In [7]:
import streamlit as st
import pandas as pd
import numpy as np
import joblib

# --- 1. Load Artifacts (Cached for speed) ---
@st.cache_resource
def load_deployment_artifacts():
    """Loads all saved model and preprocessing objects."""
    try:
        model = joblib.load('log_reg_model.pkl') 
        scaler = joblib.load('scaler.pkl')
        ord_enc = joblib.load('ordinal_encoder.pkl')
        lab_enc = joblib.load('label_encoder.pkl')
        final_columns = joblib.load('final_columns.pkl')
        return model, scaler, ord_enc, lab_enc, final_columns
    except FileNotFoundError as e:
        st.error(f"Error: Missing deployment artifact file: {e}. Please ensure all .pkl files are in the same directory.")
        return None, None, None, None, None

model, scaler, ord_enc, lab_enc, final_columns = load_deployment_artifacts()

# --- 2. Preprocessing Function ---
def preprocess_input(input_data, scaler, ord_enc, final_columns):
    """Applies the full preprocessing pipeline to the single customer input."""
    df = pd.DataFrame([input_data])
    df['TotalCharges'] = pd.to_numeric(
        df['TotalCharges'].astype(str).str.replace(' ', ''), errors='coerce'
    ).fillna(0) 
    
    if 'customerID' in df.columns:
        df.drop(columns=['customerID'], inplace=True)
    
    ordinal_cols = ['Contract']
    if ordinal_cols[0] in df.columns:
        # We must reshape the data for the ordinal encoder transformation
        df[ordinal_cols] = ord_enc.transform(df[ordinal_cols].values.reshape(-1, 1))

    df = pd.get_dummies(df, dtype='int')
    
    columns_to_drop = ['gender_Male', 'PaperlessBilling_Yes']
    features_to_drop = [col for col in columns_to_drop if col in df.columns]
    df.drop(columns=features_to_drop, inplace=True)

    missing_cols = set(final_columns) - set(df.columns)
    for c in missing_cols:
        df[c] = 0
    X_processed = df[final_columns].copy()

    continuous_features = ['TotalCharges', 'tenure', 'MonthlyCharges']
    X_processed[continuous_features] = scaler.transform(X_processed[continuous_features])
    
    return X_processed

# --- 3. Prediction Function ---
def predict_churn(X_processed, model, lab_enc):
    """Makes a prediction and inverse transforms the result."""
    churn_probability = model.predict_proba(X_processed)[:, 1][0]
    churn_prediction_encoded = model.predict(X_processed)[0]
    churn_prediction_label = lab_enc.inverse_transform([churn_prediction_encoded])[0]
    
    return churn_prediction_label, churn_probability

# --- 4. Streamlit UI ---
def main():
    st.set_page_config(
        page_title="Telco Churn Predictor", 
        layout="wide",
        initial_sidebar_state="expanded"
    )
    
    # Custom CSS for better look (optional but nice)
    st.markdown("""
        <style>
        .stButton>button {
            background-color: #f63366;
            color: white;
            font-size: 18px;
            font-weight: bold;
            border-radius: 8px;
            padding: 10px 20px;
        }
        .prediction-result {
            padding: 20px;
            border-radius: 10px;
            text-align: center;
        }
        </style>
        """, unsafe_allow_html=True)
    
    st.title("📞 Customer Churn Prediction Dashboard")
    st.markdown("Use the Tuned Logistic Regression Model (AUC **0.8287**) to predict customer retention.")
    st.markdown("---")
    
    if model is None:
        return 

    # --- Sidebar for General/Financial Inputs ---
    with st.sidebar:
        st.header("👤 Customer & Account Details")
        
        # Helper function for radio buttons
        def radio_input(label, options, key):
            return st.radio(label, options, key=key, horizontal=True)

        # Demographics
        gender = radio_input("Gender", ['Male', 'Female'], 'gender')
        SeniorCitizen = radio_input("Senior Citizen", [0, 1], 'SeniorCitizen')
        Partner = radio_input("Has Partner", ['Yes', 'No'], 'Partner')
        Dependents = radio_input("Has Dependents", ['Yes', 'No'], 'Dependents')
        
        st.markdown("---")
        st.header("💰 Financials & Contract")
        tenure = st.slider("Tenure (Months)", min_value=1, max_value=72, value=12)
        Contract = st.selectbox("Contract Type", ['Month-to-month', 'One year', 'Two year'])
        
        col_m, col_t = st.columns(2)
        with col_m:
             MonthlyCharges = st.number_input("Monthly Charges ($)", min_value=18.0, max_value=120.0, value=70.0, step=0.01)
        with col_t:
             TotalCharges = st.number_input("Total Charges ($)", min_value=0.0, value=100.0, step=0.01)

        PaperlessBilling = radio_input("Paperless Billing", ['Yes', 'No'], 'PaperlessBilling')
        PaymentMethod = st.selectbox("Payment Method", ['Electronic check', 'Mailed check', 'Bank transfer (automatic)', 'Credit card (automatic)'])


    # --- Main Area for Service Inputs ---
    st.header("📡 Service Selection")
    
    col_internet, col_security, col_tv = st.columns(3)
    
    with col_internet:
        st.subheader("Internet & Phone")
        PhoneService = radio_input("Phone Service", ['Yes', 'No'], 'PhoneService')
        MultipleLines = radio_input("Multiple Lines", ['No phone service', 'Yes', 'No'], 'MultipleLines')
        InternetService = st.selectbox("Internet Type", ['DSL', 'Fiber optic', 'No'])

    with col_security:
        st.subheader("Security & Backup")
        OnlineSecurity = radio_input("Online Security", ['No internet service', 'Yes', 'No'], 'OnlineSecurity')
        OnlineBackup = radio_input("Online Backup", ['No internet service', 'Yes', 'No'], 'OnlineBackup')
        DeviceProtection = radio_input("Device Protection", ['No internet service', 'Yes', 'No'], 'DeviceProtection')

    with col_tv:
        st.subheader("Streaming & Support")
        TechSupport = radio_input("Tech Support", ['No internet service', 'Yes', 'No'], 'TechSupport')
        StreamingTV = radio_input("Streaming TV", ['No internet service', 'Yes', 'No'], 'StreamingTV')
        StreamingMovies = radio_input("Streaming Movies", ['No internet service', 'Yes', 'No'], 'StreamingMovies')

    st.markdown("---")
    
    # --- Prediction Button and Logic ---
    if st.button('**ANALYZE CHURN RISK**'):
        # Create input dictionary
        input_dict = {
            'customerID': ['N/A'], 'gender': gender, 'SeniorCitizen': SeniorCitizen, 'Partner': Partner, 
            'Dependents': Dependents, 'tenure': tenure, 'PhoneService': PhoneService, 
            'MultipleLines': MultipleLines, 'InternetService': InternetService, 'OnlineSecurity': OnlineSecurity, 
            'OnlineBackup': OnlineBackup, 'DeviceProtection': DeviceProtection, 'TechSupport': TechSupport,
            'StreamingTV': StreamingTV, 'StreamingMovies': StreamingMovies, 'Contract': Contract, 
            'PaperlessBilling': PaperlessBilling, 'PaymentMethod': PaymentMethod, 
            'MonthlyCharges': MonthlyCharges, 'TotalCharges': TotalCharges
        }

        # Preprocess and Predict
        try:
            X_processed = preprocess_input(input_dict, scaler, ord_enc, final_columns)
            prediction, probability = predict_churn(X_processed, model, lab_enc)
        except Exception as e:
            st.error(f"Prediction Error: Could not process input. Details: {e}")
            return

        # Display Results
        st.header("Prediction Analysis")
        
        prob_churn_percent = probability * 100
        prob_retain_percent = 100 - prob_churn_percent
        
        col_res1, col_res2 = st.columns([1, 2])
        
        with col_res1:
            if prediction == 'Yes':
                st.metric(label="Predicted Status", value="CHURN", delta=f"{prob_churn_percent:.1f}% Risk", delta_color="inverse")
            else:
                st.metric(label="Predicted Status", value="RETAIN", delta=f"{prob_retain_percent:.1f}% Chance", delta_color="normal")
        
        with col_res2:
            st.markdown("#### Probability Breakdown")
            
            # Use color for visual emphasis on the progress bar
            progress_color = 'red' if prediction == 'Yes' else 'green'

            st.markdown(f"**Retention Probability:** **<span style='color:green;'>{prob_retain_percent:.1f}%</span>**", unsafe_allow_html=True)
            st.progress(prob_retain_percent / 100)

            st.markdown(f"**Churn Probability:** **<span style='color:red;'>{prob_churn_percent:.1f}%</span>**", unsafe_allow_html=True)
            st.progress(prob_churn_percent / 100)

        st.markdown("---")
        with st.expander("Show Raw Input Data", expanded=False):
            st.dataframe(pd.DataFrame([input_dict]), use_container_width=True)

if __name__ == '__main__':
    if model:
        main()

2025-11-08 22:29:10.673 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-08 22:29:10.675 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-08 22:29:10.676 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-08 22:29:10.677 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-08 22:29:10.678 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-08 22:29:10.679 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-08 22:29:10.680 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-08 22:29:10.681 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

2025-11-08 22:29:10.743 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-08 22:29:10.744 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-08 22:29:10.745 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-08 22:29:10.746 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-08 22:29:10.747 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-08 22:29:10.748 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-08 22:29:10.748 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-08 22:29:10.751 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

2025-11-08 22:29:10.796 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-08 22:29:10.798 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-08 22:29:10.801 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-08 22:29:10.802 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-08 22:29:10.802 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-08 22:29:10.803 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-08 22:29:10.803 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-08 22:29:10.805 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [8]:
import warnings as w 
w.filterwarnings('ignore')